
---
<big><big><big><big><big><big>Sieci neuronowe 2018</big></big></big></big></big></big>

---





---
<big><big><big><big>Metody regularyzacji</big></big></big></big></big>

---



---

<id=tocheading><big><big><big><big>Spis treści</big></big></big></big>
<div id="toc"></div>

---

In [ ]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

In [ ]:
# -*- coding: utf-8 -*-

import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter

plt.style.use("fivethirtyeight")

from bokeh.io import gridplot, output_file, show
from bokeh.plotting import figure, output_notebook
from bkcharts import Scatter

In [ ]:
output_notebook()

In [ ]:
sns.set(font_scale=2.0)

Image inclusion
<img src="../nn_figures/" width="100%">

# Batch normalization
1. uczenie w mini-batchach
  * mini-batche polepszają ewaluację gradientu
  * uczenie szybsze niż przy pojedynczych przykładach
2. uczenie sgd wymaga dopasowania wielu parametrów
3. __covariate shift__
  * sieć ma wiele warstw
  * rozkład danych dla warstwy wejściowej jest stały (dla danego zbioru)
  * w trakcie uczenia rozkład dla warstw kolejnych zmienia się
  * ta zmiana tym bardziej im głębsza jest sieć
  * wiele kolejnych warstw to jak złozenie funkcji, przy czym wewnątrzna __zmienia się__ w trakcie uczenia 
4. niech warstwa $Z$ ma wejście z warstwy $Y$: $$z=g(w^Ty+b)$$ a $g$ sigmoidalna
  * wzrost wartości bezwzględnej $|w^T+b|$ prowadzi $g()$ do obszaru o niskim gradiencie
  * spowoduje to ruch wielu wymiarów w kierunku saturacji
5. użycie ReLU, dobrej inicjalizacji, małych prędkości może pomóc
6. BN
  * może pozwolic na uzycie nieliniowości sigmoidalnych
  * powinno przyspieszyć uczenie
    * pojedyncze epoki są wolniejsze, ale zbieżność szybsza
    * możliwe większe prędkosci uczenia
    * większa odporność na niedobrą inicjalizację
    * zmniejszy wpływ zanikającego gradientu
    * pozwala na użycie większej liczby rodzajów funkcji aktywacji
    * ułatwia projektowanie sieci
    * jest rodzajem generalizacji

## Założenia
1. dawno wskazywano, że uczenie jest szybsze gdy wejścia są ___whitened___ (LeCun)
  * mają zerowe średnie
  * jednostkowe wariancje
2. dość proste do realizacji dla danych wejściowych (chociaż pracochłonne)
  * niech __wszystkie__ wejścia mają dodatnie wartości
  * wszystkie wagi neuronów w pierwszej warstwie będą się __wspólnie__ zwiększać lub zmniejszać
  * ścieżka do minimum będzie powolna 
  * kroki przetwarzania wejść
    * usunięcie średnich
    * dekorelacja (np. PCA)
    * wyrównanie kowariancji by dla różnych wejść były w przybliżeniu równe
    * byłoby fajnie dla kazdej warstwy
3. inicjalizacja
  * wrócimy do tego!!!!!!!!!!
1. pełna normalizacja jest trudna __dla wszystkich cech__
  * niech $x$ będzie wyjściem z danej warstwy, wtedy normalizacja
  $$\widehat{x}=Norm(x,X)$$
  gdzie $X$ jest całym zbiorem
  * to wymaga policzenia dla wstecznej propagacji
  $$\frac{\partial\,Norm(x,X)}{\partial\,x}\hskip{2em}\text{oraz}\hskip{2em}\frac{\partial\,Norm(x,X)}{\partial\,x}$$
  * potem macierzy kowariancji $Cov[x]$ oraz $Cov[x]^{-1/2}$
  * to jest co najmniej pracochłonne

## Batch Normalization
1. ___uproszczenie___ normalizacja dla każdej $k$-tej cechy osobno
$$\widehat{x}_k=\frac{x_k-E[x_k]}{\sqrt{Var[x_k]}}$$
  * przyspiesza nawet jeśli cechy są od siebie zależne
2. BN zmienia postać funkcji uczonej przez warstwę przyjmującą $x$
  * aby __warstwa BN__ potrafiła wykonywać identyczność uczymy
  $$y_k=\gamma_k\widehat{x}_k+\beta_k$$
3. ___uproszczenie___ każdy __mini-batch__ estymuje średnią i wariancję każdej aktywacji
  * efektywniejsze niż dla całego zbioru
    * psuje efekt SGD
  * mini-batch często (zwykle) jest mniejszy niż liczba cech
    * obliczanie pełnej kowariancji wymagałoby regularyzacji
4. Obliczenia dla mini-batchu $B$ o $m$ przykładach (dla pojedynczej cechy $i$)
$$\begin{align}
\mu_B&=\frac{1}{m}\sum_i^mx_i\\
\sigma_B^2&=\frac{1}{m}\sum_i^m(x_i-\mu_B)\\
\widehat{x}_i&=\frac{x_i-\mu_B}{\sqrt{\sigma_B^2+\epsilon}}\\
y_i&=\gamma\widehat{x}_i+\beta
\end{align}$$
  * końcową transformacją BN() jest $$y_k=BN_{\gamma, \beta}(x_k)=\gamma_k\widehat{x}_k+\beta_k$$
  gdzie $y_k$ są wyjściami z warstwy BN
  * wartości $\widehat{x}_i$ są __wewnątrz__ warstwy BN, ale to ich cechy są istotne
    * $\widehat{x}_i$ mają średnią $0$ i wariancję $1$
    * wyjście $y$ staje się wejściem do liniowej sieci $\gamma\widehat{x}_i+\beta$
    * potem następują kolejne warstwy oryginalnej sieci

### Batch Normalization a SGD
1. użycie BN wymaga uwzglednienia w algorytmie uczenia
  * transformacje są proste, gradienty proste do wyliczenia
  * możliwe do uczenia
  * transformacja liniowa - sieć zachowuje swoją pojemność
2. przekształcenia BN __nie są__ potrzebne w trakcie inferencji
  * po fazie uczenia stosujemy w warstwie BN normalizację __na całym zbiorze uczącym__
  $$\widehat{x}=\frac{x-E[x]}{\sqrt{Var[x]}}$$
3. cały algorytm
  1. uczenie warstwy BN
  2. dodanie transformacji $y_k=BN_{\gamma_k,\beta_k}(x_k)$
  3. użycie $y_k$ zamiast $x_k$
  4. uczenie calej sieci
  5. w sieci inferencji chwilowo obliczone parametry $\gamma,\beta$
  6. po zakończeniu uczenia uśrednianie (moving window) parametrów po wielu mini-batchach
  $$E[x]=E_B[\mu_B]\hskip{2em}Var[x]=\frac{m}{m-1}E_B[\sigma_B^2]$$
  7. w warstwie BN zastąpienie $y=BN_{\gamma,\beta}$ przez
  $$y=\frac{\gamma}{\sqrt{Var[x]+\epsilon}}+\left(\beta-\frac{\gamma\,E[x]}{Var[x]+\epsilon}\right)$$


## Gdzie umieścić BN?
1. bezpośrednio przed obliczeniem nieliniowej aktywacji
  * normalizujemy $x=w^Tu+b$, by z tego policzyć aktywację $g()$
  * właściwie _bias_ może być ominięty, ponieważ normalizacja go zniweluje odejmując średnią
2. BN można też wykorzystać w sieciach konwolucyjnych
  * trzeba zadbać, by te same cechy w różnych miejscach były normalizowane w ten sam sposób

### Zalety
1. BN pozwala na wyższe współczynniki uczenia
  * zwykle wysokie współczynniki uczenia prowadzą do wybuchu lub też zanikania gradientów
  * BN zabezpiecza sieć przed utknięciem w rejonie saturacji
2. BN regularyzuje model
  * każdy przykład jest uczony w kontekscie swojego mini-batchu
  * sieć nie zwraca deterministycznych wartości dla przykładów
  * Dropout może być usunięty albo osłabiony
    * różne są zdania
  * można zredukować regularyzację L2
3. przyspiesza uczenie
  * zbieżność jest szybsza, co nadrabia z przewagą dodatkowe obliczenia
4. potrzebne lepsze mieszanie przykładów

# Dropout 
1. pojedyncze neurony uczą się zbyt dokładnie (mogą się uczyć)
  * overfitting
2. sieć tworzy czasem zbyt dużą współzależność grup neuronów
  * pary neuronów specjalizują się w rozpoznawaniu bardzo specyficznych problemów
  * dropout __losowo__ wyłącza neurony
  * neurony mają małą szansę na powtórzenie konfiguracji
3. pozwala na uczenie badziej zgrubnych cech
  * po wyłączeniu jedych, inne neurony muszą zastąpić tamte w rozpoznawaniu
  * można zastosować dla neuronów wejściowych
4. końcowy model jest __ensemblem__, przy czym liczba tworzonych modeli jest olbrzymia
  * przy wyłączaniu tworzy się wiele niezaleźnych wewnętrznych reprezentacji
5. zwykle wymaga więcej epok uczenia
6. niekoniecznie dobrze współpracuje z BN

## Dropout - założenia
1. Nitish Srivastava, Geoffrey Hinton, Alex Krizhevsky, Ilya Sutskever, Ruslan Salakhutdinov
2. najlepszym sposobem, przy nieograniczonych zasobach, będzie
  * uśrednić predykcje po wszystkich ustawieniach parametrów
  * ważyć każde przez posterior prawdopodobieństwa przy zadanych danych
  * jest to zwykle możliwe tylko dla bardzo małych modeli
3. przy łączeniu modeli w ensemble ważne by modele były __różne__ (ang. diverse)
  * modele są różne, jeśli _popełniają błędy w różnych miejscach_
  * uczenie sieci jest kosztowne
4. Dropout stara się rozwiazać oba problemy
  * czasowo odrzucane są poszczególne neurony
    * widzialne lub ukryte
  * prawdopodobieństwo dla ukrytych jest zwykle około $0.5$
  * dla wejściowych nawet bliżej $1$!
  * sieć staje się bardzo rozrzedzona
5. pełna sieć to zbiór $2^n$ różnych zubożonych modeli
  * dzielą jednak parametry
  * połączenie słabych modeli może dawać model silny
6. w jaki sposób ewoluować model?
  * nie da się obliczyć wszystkich
  * można wziąć jeden cały model
    * wagi tego modelu są skalowanymi wersjami modelu pełnego
    * można przemnożyć wagi przez prawdopodobieństwo, że waga była uzytwa w trakcie uczenia
7. Dropout jest metodą regularyzacji przez dodanie szumu
  * autoencoder dostaje zaszumione dane
  * zadaniem jest je odtworzyć

## Dropout
1. sieć warstwowa
$$\begin{align}
\textbf{bez dropout}\hskip{2em}&\hskip{2em}\textbf{z dropout}\\
\hskip{2em}&\hskip{2em}r\sim{}Bernoulli(p)\\
\hskip{2em}&\hskip{2em}\widetilde{y}^{(l)}=r*y^{(l)}\\
z_i^{(l)}=w_i^{(l+1)}\,y^{(l)}+b_i^{(l+1)}\hskip{2em}&\hskip{2em}z_i^{(l)}=w_i^{(l+1)}\,\widetilde{y}^{(l)}+b_i^{(l+1)}\\
y_i^{(l+1)}=g(z_i^{(l+1)})\hskip{2em}&\hskip{2em}y_i^{(l+1)}=g(z_i^{(l+1)})
\end{align}$$
2. uczenie
  * dowolny algorytm typu SGD
  * po wylosowaniu neuronów wsteczna propagacja __wyłącznie__ dla pozostawionych
  * ważną formą regularyzacji jest __ograniczenie__ normy wektora wag wchodzącego do dowolnego neurona do ustalonej wartości
    * tzw. __max-norm__ przez ograniczenie wektora do kuli o promieniu $c$
  * także wysokie wartości __decay__ przyspieszają zbieżność
3. Odtwarzanie
  * wybranie modelu i przemnożenie wag
  * ensemble
    * wybrać $k$ modeli
      * dla każdego postąpić znaleźć predykcje
      * wszystkie wyniki uśrednić
      * dla MNIST ok. $50$ modeli równa się z pełnym mnożonym
      * zwykle wystarczy wziąć ok. $10$ niezależnych predykcji
    * bardziej kosztowne
4. Cechy uczenia
  * uczenie wymaga zwykle sporo większej liczby epok
  * neurony ukryte mają rzadką aktywację
  * parametr prawdopodobieństwa pozostawienie neuronu $p$ można wybrać
    * niskie $p$ (mało neuronów) powoduje niedofitowanie,  (dla MNIST)
    * zwykle $p\geq0.4$ daje najlepsze wyniki